# Homework 1

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

########## Graph-Search-Related Utilities and Class Definitions ##########

from collections import namedtuple, defaultdict
from typing import Optional, List
import heapq as hq
import numpy as np
import math
import random


def grid_successor_fn(state):
  """Helper for testing heuristic search.
  """
  arrival_costs = np.array([
    [1, 1, 8, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 1, 1, 8, 1],
    [1, 1, 2, 1, 1],
  ], dtype=float)

  act_to_delta = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
  }

  r, c = state

  for act, (dr, dc) in act_to_delta.items():
    new_r, new_c = r + dr, c + dc
    # Check if in bounds
    if not (0 <= new_r < arrival_costs.shape[0] and \
            0 <= new_c < arrival_costs.shape[1]):
      continue
    # Valid action
    yield (act, (new_r, new_c), arrival_costs[new_r, new_c])


def grid_check_goal_fn(state):
  """Helper for testing heuristic search.
  """
  # Bottom right corner of grid
  return state == (4, 4)


def grid_heuristic_fn(state):
  """Helper for testing heuristic search.
  """
  # Manhattan distance
  return abs(state[0] - 4) + abs(state[1] - 4)


# A useful data structure for heuristic search
Node = namedtuple("Node", ["state", "parent", "action", "cost", "g"])




## Problems

### Heuristic Search
Complete an implementation of heuristic search, encompassing A*, GBFS, or UCS.  You can assume any heuristics are consistent.

For reference, our solution is **46** lines of code.

In [ ]:
def run_heuristic_search(initial_state, check_goal, get_successors, get_priority, max_expansions=1000):
  """A generic heuristic search implementation.

  Depending on get_priority, can implement A*, GBFS, or UCS.

  The get_priority function here should determine the order
  in which nodes are expanded. For example, if you want to
  use path cost as part of this determination, then the
  path cost (node.g) should appear inside of get_priority,
  rather than in this implementation of `run_heuristic_search`.

  Important: for determinism (and to make sure our tests pass),
  please break ties using the state itself. For example,
  if you would've otherwise sorted by get_priority(node), you
  should now sort by (get_priority(node), node.state).

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    get_priority: A callable that takes a Node and returns a
        float priority, with lower better, for the priority
        queue. This function is what switches between different
        versions of heurstic search.
    max_expansions: An int maximum number of nodes to expand
        before giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# We will test this implementation more thoroughly with the
# specific heuristic search algorithms that follow
initial_state = (0, 0)
get_priority_fn = lambda node: 0
result = run_heuristic_search(initial_state, grid_check_goal_fn,
    grid_successor_fn, get_priority_fn)
assert len(result) == 4

def heuristic_search_test2():
    # We will test this implementation more thoroughly with the
    # specific heuristic search algorithms that follow
    initial_state = (0, 0)
    get_priority_fn = lambda node: 0
    state_sequence, action_sequence, cost_sequence, num_expansions = run_heuristic_search(initial_state, grid_check_goal_fn,
        grid_successor_fn, get_priority_fn)
    # Textbook implementation
    try:
        assert state_sequence == [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
        assert action_sequence == ['right', 'right', 'right', 'right', 'down', 'down', 'down', 'down']
        assert cost_sequence == [1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
    # Alternative implementation that tracks best-cost-to-nodes
    except AssertionError:
        assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
        assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
        assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
    assert num_expansions <= 35

heuristic_search_test2()
def heuristic_search_test3():
    # If your results do not match the expected ones, make sure that you are tiebreaking
    # as described in the docstring for `run_heuristic_search`.
    initial_state = (0, 0)
    get_priority_fn = lambda node: node.g
    state_sequence, action_sequence, cost_sequence, num_expansions = run_heuristic_search(initial_state, grid_check_goal_fn,
        grid_successor_fn, get_priority_fn)
    assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
    assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
    assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
    assert num_expansions <= 35

heuristic_search_test3()
print('Tests passed.')

### Uniform Cost Search
Use your implementation of `run_heuristic_search` to implement uniform cost search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_uniform_cost_search(initial_state, check_goal, get_successors, max_expansions=1000):
  """Uniform-cost search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def ucs_test1():
    # If your results do not match the expected ones, make sure that you are tiebreaking
    # as described in the docstring for `run_heuristic_search`.
    initial_state = (0, 0)
    state_sequence, action_sequence, cost_sequence, num_expansions = run_uniform_cost_search(
        initial_state, grid_check_goal_fn, grid_successor_fn)
    assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
    assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
    assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
    assert num_expansions <= 30

ucs_test1()
print('Tests passed.')

### A* Search
Use your implementation of `run_heuristic_search` to implement A* search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_astar_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """A* search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def astar_test1():
    initial_state = (0, 0)
    state_sequence, action_sequence, cost_sequence, num_expansions = run_astar_search(
        initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
    assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
    assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
    assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
    assert num_expansions <= 20

astar_test1()
print('Tests passed.')

### Greedy Best-First Search
Use your implementation of `run_heuristic_search` to implement GBFS.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_greedy_best_first_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """GBFS.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def gbfs_test1():
    # If your results do not match the expected ones, make sure that you are tiebreaking
    # as described in the docstring for `run_heuristic_search`.
    initial_state = (0, 0)
    state_sequence, action_sequence, cost_sequence, num_expansions = run_greedy_best_first_search(
        initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
    assert state_sequence == [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
    assert action_sequence == ['right', 'right', 'right', 'right', 'down', 'down', 'down', 'down']
    assert cost_sequence == [1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
    assert abs(num_expansions - 8) <= 1

gbfs_test1()
print('Tests passed.')